In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import StepLR

# Data loading and processing

In [ ]:
#load csv files to pandas dataframe
train_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)

In [ ]:
#the last column is the data label described as category from 0 to 4
#so we can use it as a target
y_df = train_df.iloc[:, -1]
y_df = y_df.astype('int')
y_df.head()

In [ ]:
#the rest are the ecg datapoints so we use them as features
X_df = train_df.iloc[:, :-1]
X_df.head()

In [ ]:
#convert all feautures to float type
X_df = X_df.astype('float')
X_df.head()

In [ ]:
#we actually dont need to one hot encode the target since we are using cross entropy loss and it runs more efficiently when we pass in the target as indices
#print out the unique values in the target
unique, counts = np.unique(y_df, return_counts=True)
print(f'unique values: {unique}')
print(f'counts: {counts}')

In [ ]:
#make a dictionary to map the target indices to class names
class_names = {0: 'N', 1: 'S', 2: 'V', 3: 'F', 4: 'Q'}

In [ ]:
#plot the first 5 ecg signals in the dataset and their corresponding labels 
fig, ax = plt.subplots(5, 1, figsize=(20, 10))
for i in range(5):
    ax[i].plot(X_df.iloc[i, :])
    ax[i].set_title(f'Label: {class_names[y_df[i]]}')
    ax[i].set_xlabel('Time (ms)')
    ax[i].set_ylabel('Amplitude (mV)')

In [ ]:
#the input dims are 187 and the output dims are 5
#we want the input to be (batch_size, 1, 187) and the output to be (batch_size, 5) because the input has one channel and the output has 5 classes
X_np = X_df.to_numpy()
X_np.shape

In [ ]:
#we want to add a channel dimension to the input
X_np = X_np.reshape(-1, 1, 187)
X_np.shape

In [ ]:
#do the same for the target
y_np = y_df.to_numpy()
y_np.shape

In [ ]:
y_np = y_np.reshape(-1, 1)
y_np.shape

# Data normalization

In [ ]:
#perfect, we now have the input and target in the correct shape
X_mean = X_np.mean()
X_std = X_np.std()
X_max = X_np.max()
X_min = X_np.min()
print(f'mean before normalization: {X_mean}')
print(f'std: {X_std}')
print(f'max: {X_max}')

In [ ]:
#we need to subtract the mean from the data
X_norm = X_np - X_mean
#then multiply by 1/std to make the std = 1
X_norm *= 1/X_std

In [ ]:
print(X_norm.shape)
print(f'new mean is {X_norm.mean()}')
print(f'new std is {X_norm.std()}')

In [ ]:
#now plot the normalized data
fig, ax = plt.subplots(5, 1, figsize=(20, 10))
for i in range(5):
    ax[i].plot(X_norm[i, 0, :])
    ax[i].set_title(f'Label: {class_names[y_df[i]]}')
    ax[i].set_xlabel('Time (ms)')
    ax[i].set_ylabel('Amplitude (mV)')

In [ ]:
#now plot the distribution of data 
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
ax.hist(X_norm.flatten(), bins=100)
ax.set_title(f'Distribution of data')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (mV)')

In [ ]:
X_norm.shape #representing the input data as a sequence of 187 points

In [ ]:
y_np.shape #representing the target as indices 0 to 4

# Saving the normalized variables

In [ ]:
#saving the normalized data to a csv file
np.savetxt('X_norm.csv', X_norm.squeeze(), delimiter=',')
np.savetxt('y_np.csv', y_np, delimiter=',')
# np.loadtxt('X_norm.csv', delimiter=',').shape

# Loading the normalized variables from disk

In [ ]:
X_norm = np.loadtxt('X_norm.csv', delimiter=',').reshape(-1, 1, 187)
y_np = np.loadtxt('y_np.csv', delimiter=',').reshape(-1, 1)

In [ ]:
print(X_norm.shape)
print(y_np.shape)

In [ ]:
#change y_np data type to int
y_np = y_np.astype('int')
y_np.dtype

## We split the data into training and validation sets using scikit-learn

In [ ]:
#shuffle the data and split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_norm, y_np, test_size=0.2, shuffle=True)


### Now we check for class imbalance

In [ ]:
#print the frequency of each class in the train and validation sets
print(f'Frequency of each class in the train set')
unique, counts = np.unique(y_train, return_counts=True)
for i in range(len(unique)):
    print(f'{class_names[unique[i]]}: {counts[i]}')

print(f'Frequency of each class in the validation set')
unique, counts = np.unique(y_val, return_counts=True)
for i in range(len(unique)):
    print(f'{class_names[unique[i]]}: {counts[i]}')

### We define a hyper-parameters dictionary

In [ ]:
hyper_params = {'bs': 1024,
                                'lr': 1e-3,
                                'lr_decay': 0.3,
                                'epochs':50,
                                }

In [ ]:
#pytorch dataset and dataloader
train_ds = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_ds = torch.utils.data.TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

In [ ]:
len(train_ds), len(val_ds)

### We compute the weight of each class by inverting their distribution and normalizing it

In [ ]:
#the class distribution is not uniform so we have to define weights for each class
class_dist = np.unique(y_train, return_counts=True)[1] #index 1 returns the counts
class_dist = class_dist/np.sum(class_dist) #normalize to get 0-1 values
# normalize the weights
weights = 1/class_dist #invert to compensate for class distribution imbalance
weights = weights/weights.sum() #normalize to get values 0-1
weights, class_names

### We assign each example in the training set the weight we computed in the above cell

In [ ]:
#get weights for each example in the dataset
data_weights = np.zeros(y_train.shape[0]) #shape (m, )
for i in range(y_train.shape[0]): #for each example we assign the corresponding weight
    data_weights[i] = weights[y_train[i, 0]]
data_weights[:10]

### Then we pass the weights to the WeightedRandomSampler to sample according to the normalized weights

In [ ]:
#data loader - we use the random weights sampler to balance the classes
train_dl = DataLoader(train_ds, batch_size=hyper_params['bs'], sampler=WeightedRandomSampler(weights=data_weights, num_samples= len(y_train), replacement=True))
val_dl = DataLoader(val_ds, batch_size=hyper_params['bs'])


#### We check the training dataloader for class distributions

In [ ]:
#get the class distribution in dataloader
for batch in train_dl:
    print(batch[1].shape)
    unique, counts = np.unique(batch[1], return_counts=True)
    for i in range(len(unique)):
        print(f'{class_names[unique[i]]}: {counts[i]}')
    break

We can see that the distribution is nearly uniform now

In [ ]:
#sample random batch from the train dataloader and plot the first 5 ecg signals in the batch
x, y = next(iter(train_dl))
fig, ax = plt.subplots(5, 1, figsize=(20, 10))
for i in range(5):
    ax[i].plot(x[i, 0, :])
    ax[i].set_title(f'Label: {class_names[y[i].item()]}')
    ax[i].set_xlabel('Time (ms)')
    ax[i].set_ylabel('Amplitude (mV)')

# Network Architecture
### Here, we define a Network-in-network arch to process signals
### instead of using linear layers at the end of the model to classify the signals, we use 1x1 Conv1d layers to capture correlations and global average pooling and reduce the dims
### Also, while experimenting I found out that GELU activation works better than RELU in this case

In [ ]:
#make some convolutional blocks to use in the model
def nin_block(in_channels, out_channels, kernel_size, padding, strides):
    return nn.Sequential(
        nn.Conv1d(in_channels, out_channels, kernel_size, strides, padding),
        nn.BatchNorm1d(out_channels),
        nn.GELU(),
        nn.Conv1d(out_channels, out_channels, kernel_size=1), nn.GELU(),
        nn.Conv1d(out_channels, out_channels, kernel_size=1), nn.GELU()
    )

In [ ]:
def get_model():
    return nn.Sequential( #input shape: (batch_size, 1, 187)
        nin_block(1, 48, kernel_size=11, strides=4, padding=0), #output shape: (batch_size, 48, 44)
        nn.MaxPool1d(3, stride=2), #output shape: (batch_size, 48, 21)
        nin_block(48, 128, kernel_size=5, strides=1, padding=2), #output shape: (batch_size, 128, 21)
        nn.MaxPool1d(3, stride=2), #output shape: (batch_size, 128, 10)
        nin_block(128, 256, kernel_size=3, strides=1, padding=1), #output shape: (batch_size, 256, 10)
        nn.MaxPool1d(3, stride=2), #output shape: (batch_size, 256, 4)
        nn.Dropout(0.4), 
        #last layers for classification of 5 classes
        nin_block(256, 5, kernel_size=3, strides=1, padding=1), #output shape: (batch_size, 5, 4)
        nn.AdaptiveAvgPool1d(1), #output shape: (batch_size, 5, 1)
        nn.Flatten() #output shape: (batch_size, 5)
    )

In [ ]:
model = get_model()

In [ ]:
#model size in MB
print(f'Model size: {sum(p.numel() for p in model.parameters() if p.requires_grad)*4/(1024**2)} MB')
#number of trainable parameters
print(f'Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

In [ ]:
#find device to run the model on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
#move the model to the device
model = model.to(device)

In [ ]:
class_names

While experimenting, I found out that if I set some weights to some classes the model performs better.

This way I force the model to learn the classes that it didn't do well on.

In [ ]:
#loss function
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1, 1.1, 2.5, 3, 1], dtype=torch.float, device=device)) #CE loss automatically uses indices as targets

In [ ]:
#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['lr'])

In [ ]:
#lr_scheduler
lr_scheduler = StepLR(optimizer, step_size=10, gamma=hyper_params['lr_decay'], verbose=True)

### Training function

In [ ]:
def train(model, criterion, optimizer, train_dl, val_dl, epochs, lr_scheduler: StepLR=None):
    """trains the model for the given number of epochs

    Args:
        model (nn.Module): nn model
        criterion (nn.CELoss): loss function
        optimizer (torch.optim): optimizer
        train_dl (Dataloader): training dataloader
        val_dl (Dataloader): validation dataloader
        epochs (int): number of epochs to train for
        lr_scheduler (StepLR): learning rate scheduler to adjust lr during training
    Returns:
        list of training losses and validation losses, best model parameters, best accuracy
    """
    train_losses = []
    val_losses = []
    best_model_params = deepcopy(model.state_dict())
    best_accuracy = 0.0
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-'*10)
        train_loss = 0
        val_loss = 0
        model.train() #set the model to training mode
        for x, y in train_dl: #x: (batch_size, 1, 187), y: (batch_size, 1)
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(x) #the output shape is (batch_size, 5) so it's a distribution over the 5 classes
            loss = criterion(output, y.squeeze())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*x.size(0)
        train_loss = train_loss/len(train_dl.dataset)
        train_losses.append(train_loss)
        lr_scheduler.step() #update the learning rate every n epochs
        model.eval() #set the model to evaluation mode
        corrects = 0 #in order to calculate accuracy we store the number of correct predictions
        for x, y in val_dl:
            x = x.to(device)
            y = y.to(device)
            output = model(x) #out shape: (batch_size, 5)
            loss = criterion(output, y.squeeze())
            val_loss += loss.item()*x.size(0)
            #calculate the number of correct predictions
            corrects += torch.sum(torch.argmax(output, dim=1) == y.squeeze()).item()
        val_loss = val_loss/len(val_dl.dataset)
        accuracy = corrects/len(val_dl.dataset)
        val_losses.append(val_loss)
        print(f'Train Loss: {train_loss:.4f} \t Val Loss: {val_loss:.4f} \t Val Accuracy: {accuracy:.4f}')
        #if the model performs better on the validation set, save the model parameters
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_params = deepcopy(model.state_dict())
    print('Finished Training and the best accuracy is: {:.4f}'.format(best_accuracy))
    return train_losses, val_losses, best_model_params, best_accuracy

In [ ]:
train_losses, val_losses, best_model_params, best_accuracy = train(model, criterion, optimizer, train_dl, val_dl, hyper_params['epochs'], lr_scheduler)

### The model finishes training with a validation accuracy of 98.6% !

In [ ]:
#plot the training and validation losses with smooth lines
plt.plot(np.convolve(train_losses, np.ones(3)/3, mode='valid'), label='train')
plt.plot(np.convolve(val_losses, np.ones(3)/3, mode='valid'), label='val')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
#load the best model parameters
model.load_state_dict(best_model_params)
#save the model to a file
torch.save(model.state_dict(), f'best_model{best_accuracy:.2f}.pt')

In [ ]:
#use this cell to load weights from disk
model = get_model().to(device=device)
model.load_state_dict(torch.load('best_model0.99.pt'))

In [ ]:
#confusion matrix function
def plot_cm(model, dl, categories, normalize='true'):
    #plot the confusion matrix
    model.eval()
    y_pred = []
    y_true = []
    for x, y in dl:
        x = x.to(device)
        y = y.to(device)
        output = model(x) #out shape: (batch_size, 5)
        y_pred.extend(torch.argmax(output, dim=1).cpu().numpy())
        y_true.extend(y.cpu().numpy())
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    sns.heatmap(cm, annot=True, fmt= '.2f', cmap='Blues', xticklabels=categories.values(), yticklabels=categories.values())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    print(classification_report(y_true, y_pred, target_names=class_names.values()))

### Let's not get too excited, we need to examine the results and make sure the model doesn't trick us by skipping minor classes

In [ ]:
#plot the confusion matrix using the validation set
plot_cm(model, val_dl, class_names)

#### We can see the model does excellently on the validation set

In [ ]:
#visualize the predictions the model got wrong
model.eval()
count = 0
for x, y in val_dl:
    x = x.to(device)
    y = y.to(device)
    output = model(x) #out shape: (batch_size, 5)
    y_pred = torch.argmax(output, dim=1)
    for i in range(len(y_pred)):
        if y_pred[i] != y[i] and count < 5:
            print(f'True: {class_names[y[i].item()]} \t Predicted: {class_names[y_pred[i].item()]}')
            plt.plot(x[i, 0, :].cpu().numpy())
            plt.show()
            count += 1

---------------------------------------------------------------------------------------------------------------------------------------------
# this section calculates the model accuracy on **test** data

In [ ]:
#this section loads the test data and calculates the accuracy on the test set
test_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)
test_df.head()

In [ ]:
#the last column is the label
label_df = test_df.iloc[:, -1]
label_df = label_df.astype('int')
label_df.head()

In [ ]:
#the rest of the columns are the signal
features_df = test_df.iloc[:, :-1]
features_df = features_df.astype('float32')
features_df.head()

In [ ]:
#convert to numpy arrays
X_test_np = features_df.to_numpy()
y_test_np = label_df.to_numpy()

In [ ]:
X_test_np.shape, y_test_np.shape

In [ ]:
#we have to reshape the data to be (batch_size, 1, 187) because that's the shape the model expects
X_test_np = np.reshape(X_test_np, (X_test_np.shape[0], 1, X_test_np.shape[1]))
X_test_np.shape

In [ ]:
y_test_np = np.reshape(y_test_np, (y_test_np.shape[0], 1))
y_test_np.shape

In [ ]:
#we have to normalize the data
X_test_np = (X_test_np - X_test_np.mean())/X_test_np.std()
X_test_np[:5, 0, :5]

In [ ]:
#visualize the first 5 samples
for i in range(5):
    plt.plot(X_test_np[i, 0])
    plt.title(class_names[y_test_np[i, 0]])
    plt.show()

In [ ]:
print(f'test mean: {X_test_np.mean():.4f} \t test std: {X_test_np.std():.4f}')

In [ ]:
#print the number of samples in each class
for i in range(5):
    print(f'class {i}: {np.sum(y_test_np == i)}')

In [ ]:
#create a dataset
test_ds = TensorDataset(torch.from_numpy(X_test_np), torch.from_numpy(y_test_np))
test_dl = DataLoader(test_ds, batch_size=hyper_params['bs'], shuffle=False)

In [ ]:
#use this cell to load weights from disk
model = get_model().to(device=device)
model.load_state_dict(torch.load('best_model0.99.pt'))

In [ ]:
#calculate the sensitivity and specificity for each class using scikit-learn
plot_cm(model, test_dl, class_names)

### We see here that the model does as well as it did on the validation set, this indicates that the model has learned to generalize!

### We have a total accuracy of 98% on the test data
#### also we have to pay attention to precision and recall
#### we don't have the numbers we had with the validation set but still it does very well